In [ ]:
from MagicCube import MagicCube
import random
import time
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def GENETICALGORITHM(initial_cube, population_size, max_generations, tournament_size=20):
    def initialize_population(size):
        return [MagicCube(5) for _ in range(size)]

    def fitnessFunction(cube): # Fitness Function
        return 1 / (1 + cube.objectiveFunction())

    def tournament_selection(population, size): # Selection menggunakan tournament
        tournament = random.sample(population, size)
        return max(tournament, key=fitnessFunction)

    def crossover(parent1, parent2): # Crossover mengugunakan metode uniform crossover
        n = parent1.cube.shape[0]
        child = MagicCube(n)
        
        mask = np.random.rand(*parent1.cube.shape) < 0.5 
        child.cube[mask] = parent1.cube[mask]
        child.cube[~mask] = parent2.cube[~mask]
        
        return child

    def repair(child): # Untuk mengurus value duplikat dan value yang hilang
        unique_numbers = np.unique(child.cube)
        all_numbers = set(range(1, child.n**3 + 1))
        duplicates = list(set(unique_numbers[unique_numbers != 0]) - all_numbers)
        missing_numbers = list(all_numbers - set(unique_numbers))

        for i in range(child.n):
            for j in range(child.n):
                for k in range(child.n):
                    if list(child.cube.flatten()).count(child.cube[i, j, k]) > 1 and missing_numbers:
                        child.cube[i, j, k] = missing_numbers.pop(0)

    def mutate(cube): # 
        num_mutations = max(1, cube.n // 5)
        for _ in range(num_mutations):
            pos1 = (random.randint(0, cube.n - 1), random.randint(0, cube.n - 1), random.randint(0, cube.n - 1))
            pos2 = (random.randint(0, cube.n - 1), random.randint(0, cube.n - 1), random.randint(0, cube.n - 1))
            cube.swapElements(pos1, pos2)
        repair(cube) 
        
    population = initialize_population(population_size) # Inisialisasi populasi
    best_fitness_history = [] # Untuk menyimpan history best fitness
    avg_fitness_history = [] # Untuk menyimpan history average fitness
    start_time = time.time() # Untuk menghitung waktu eksekusi

    for generation in range(max_generations): # Looping untuk setiap generasi
        new_population = []

        num_elites = 2 
        elites = sorted(population, key=fitnessFunction, reverse=True)[:num_elites] # Mengambil 2 individu terbaik
        new_population.extend(elites) # Menambahkan 2 individu terbaik ke new_population

        while len(new_population) < population_size: # Looping untuk mengisi new_population
            parent1 = tournament_selection(population, tournament_size)
            parent2 = tournament_selection(population, tournament_size)
            child = crossover(parent1, parent2)
            repair(child)
            mutate(child)
            new_population.append(child)

        population = new_population # Update populasi 

        fitnesses = [fitnessFunction(cube) for cube in population] # Menghitung fitness untuk setiap individu
        best_fitness = max(fitnesses) # Mengambil fitness terbaik
        avg_fitness = sum(fitnesses) / len(fitnesses) # Menghitung rata-rata fitness

        best_obj_value = elites[0].objectiveFunction() # Mengambil nilai fungsi objektif terbaik
        avg_obj_value = sum(cube.objectiveFunction() for cube in population) / len(population) # Menghitung rata-rata nilai fungsi objektif

        best_fitness_history.append(best_obj_value) 
        avg_fitness_history.append(avg_obj_value)

        print(f"Generation {generation}: Best f(x) = {best_obj_value}, Avg f(x) = {avg_obj_value}")

        if best_obj_value == 0: # Jika mencapai objective function, maka berhenti
            break

    duration = time.time() - start_time 
    best_solution = max(population, key=fitnessFunction) # Mengambil individu terbaik
    return best_solution, best_fitness_history, avg_fitness_history, generation + 1, duration # Mengembalikan individu terbaik, history best fitness, history average fitness, jumlah generasi, dan waktu eksekusi

In [ ]:
def visualize_experiment(initial_cube, final_cube, best_history, avg_history):
    print("Initial State:")
    initial_cube.visualize()

    print("Final State:")
    final_cube.visualize()

    plt.figure(figsize=(10, 6))
    plt.plot(best_history, label='Best f(x)')
    plt.plot(avg_history, label='Average f(x)')
    plt.title('Objective Function terhadap Banyak Generasi')
    plt.xlabel('Generasi')
    plt.ylabel('Objective Function (f(x))')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# Kontrol: Jumlah Populasi
if __name__ == "__main__":
    population_sizes = [100] # Jumlah Populasi
    max_generations_list = [100, 300, 500] # Banyaknya Iterasi/Generasi

    for pop_size in population_sizes:
        for max_gen in max_generations_list:
            print(f"\nEksperimen: Ukuran Populasi = {pop_size}, Maksimum Generasi = {max_gen}")
            
            for run in range(3):  # Jalankan 3 kali untuk setiap konfigurasi
                initial_cube = MagicCube(n=5)
                
                final_cube, best_history, avg_history, generations, duration = GENETICALGORITHM(initial_cube, pop_size, max_gen)

                print(f"Run {run + 1}:")
                print(f"Total Generasi: {generations}")
                print(f"Durasi: {duration:.2f} detik")
                print(f"Final Objective Function: {final_cube.objectiveFunction()}")
                
                visualize_experiment(initial_cube, final_cube, best_history, avg_history)

In [ ]:
# Kontrol: Banyaknya Iterasi (Generasi)
if __name__ == "__main__":
    population_sizes = [50, 100, 200] # Jumlah Populasi
    max_generations_list = [200] # Banyaknya Iterasi/Generasi

    for pop_size in population_sizes:
        for max_gen in max_generations_list:
            print(f"\nEksperimen: Ukuran Populasi = {pop_size}, Maksimum Generasi = {max_gen}")
            
            for run in range(3):  # Jalankan 3 kali untuk setiap konfigurasi
                initial_cube = MagicCube(n=5)
                
                final_cube, best_history, avg_history, generations, duration = GENETICALGORITHM(initial_cube, pop_size, max_gen)

                print(f"Run {run + 1}:")
                print(f"Total Generasi: {generations}")
                print(f"Durasi: {duration:.2f} detik")
                print(f"Final Objective Function: {final_cube.objectiveFunction()}")
                
                visualize_experiment(initial_cube, final_cube, best_history, avg_history)